# Deteccion de Edificios en San Juan de Pasto con IA

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/04_Deteccion_Edificios_Pasto.ipynb)

---

## Objetivos

1. Descargar huellas de edificios de Overture Maps y Google Open Buildings
2. Usar Segment Anything Model (SAM) para detectar edificios
3. Visualizar y analizar la distribucion de edificios en Pasto
4. Comparar diferentes fuentes de datos de edificios

## 1. Instalacion de Dependencias

In [ ]:
%pip install -q geoai-py leafmap overturemaps geopandas

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
import os
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")

## 3. Definicion del Area de Estudio

In [ ]:
# Coordenadas de San Juan de Pasto
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}

# Bounding Box del centro urbano de Pasto
# Area mas pequeña para demostracion
PASTO_BBOX = (-77.295, 1.200, -77.270, 1.225)

# Bounding Box extendido
PASTO_BBOX_EXTENDED = (-77.32, 1.18, -77.24, 1.26)

# Crear geometria
pasto_geometry = box(*PASTO_BBOX)

print(f"Area de estudio: {PASTO_BBOX}")
print(f"Geometria: {pasto_geometry}")

## 4. Descargar Edificios de Overture Maps

Overture Maps es un proyecto colaborativo que proporciona datos abiertos de edificios a nivel mundial.

In [ ]:
# Descargar edificios de Overture Maps
try:
    buildings_overture = geoai.download_overture_buildings(
        bbox=PASTO_BBOX, output=None  # Retorna GeoDataFrame directamente
    )
    print(f"Edificios de Overture Maps: {len(buildings_overture)}")
    print(f"\nColumnas disponibles: {list(buildings_overture.columns)}")
except Exception as e:
    print(f"Error descargando de Overture: {e}")
    buildings_overture = None

In [ ]:
# Ver primeros edificios
if buildings_overture is not None and len(buildings_overture) > 0:
    print("Primeros 5 edificios:")
    display(buildings_overture.head())

## 5. Descargar Edificios de Google Open Buildings

Google Open Buildings contiene huellas de edificios detectadas automaticamente usando IA.

In [ ]:
# Descargar edificios de Google Open Buildings
try:
    buildings_google = geoai.download_google_buildings(bbox=PASTO_BBOX, output=None)
    print(f"Edificios de Google Open Buildings: {len(buildings_google)}")
except Exception as e:
    print(f"Error descargando de Google: {e}")
    buildings_google = None

## 6. Visualizar Edificios en Mapa Interactivo

In [ ]:
# Crear mapa interactivo
m = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=15, height="600px"
)

# Agregar capas base
m.add_basemap("Esri.WorldImagery")
m.add_basemap("OpenStreetMap")

# Agregar edificios de Overture
if buildings_overture is not None and len(buildings_overture) > 0:
    m.add_gdf(
        buildings_overture,
        layer_name="Edificios Overture",
        style={
            "color": "blue",
            "fillColor": "lightblue",
            "fillOpacity": 0.5,
            "weight": 1,
        },
    )

# Agregar edificios de Google
if buildings_google is not None and len(buildings_google) > 0:
    m.add_gdf(
        buildings_google,
        layer_name="Edificios Google",
        style={"color": "red", "fillColor": "pink", "fillOpacity": 0.5, "weight": 1},
    )

# Agregar bounding box
gdf_bbox = gpd.GeoDataFrame(
    {"nombre": ["Area de Estudio"]}, geometry=[pasto_geometry], crs="EPSG:4326"
)
m.add_gdf(
    gdf_bbox,
    layer_name="Area de Estudio",
    style={"color": "green", "fillOpacity": 0, "weight": 3},
)

m.add_layer_control()
m

## 7. Analisis Estadistico de Edificios

In [ ]:
# Calcular estadisticas de edificios
def analizar_edificios(gdf, nombre):
    """
    Calcula estadisticas de un GeoDataFrame de edificios.
    """
    if gdf is None or len(gdf) == 0:
        return None

    # Asegurar que el CRS sea proyectado para calcular areas
    gdf_proj = gdf.to_crs(epsg=32618)  # UTM zona 18N para Colombia

    areas = gdf_proj.geometry.area

    return {
        "Fuente": nombre,
        "Total Edificios": len(gdf),
        "Area Total (m2)": areas.sum(),
        "Area Promedio (m2)": areas.mean(),
        "Area Minima (m2)": areas.min(),
        "Area Maxima (m2)": areas.max(),
        "Mediana Area (m2)": areas.median(),
    }


# Analizar ambas fuentes
stats = []
if buildings_overture is not None:
    stats.append(analizar_edificios(buildings_overture, "Overture Maps"))
if buildings_google is not None:
    stats.append(analizar_edificios(buildings_google, "Google Open Buildings"))

if stats:
    stats_df = pd.DataFrame([s for s in stats if s is not None])
    print("Comparacion de fuentes de edificios en Pasto:")
    display(stats_df)

In [ ]:
# Histograma de areas de edificios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if buildings_overture is not None and len(buildings_overture) > 0:
    gdf_proj = buildings_overture.to_crs(epsg=32618)
    areas = gdf_proj.geometry.area
    axes[0].hist(
        areas[areas < 500], bins=50, color="blue", alpha=0.7, edgecolor="darkblue"
    )
    axes[0].set_xlabel("Area (m2)")
    axes[0].set_ylabel("Frecuencia")
    axes[0].set_title("Distribucion de Areas - Overture Maps")
    axes[0].axvline(
        x=areas.median(),
        color="red",
        linestyle="--",
        label=f"Mediana: {areas.median():.1f} m2",
    )
    axes[0].legend()

if buildings_google is not None and len(buildings_google) > 0:
    gdf_proj = buildings_google.to_crs(epsg=32618)
    areas = gdf_proj.geometry.area
    axes[1].hist(
        areas[areas < 500], bins=50, color="red", alpha=0.7, edgecolor="darkred"
    )
    axes[1].set_xlabel("Area (m2)")
    axes[1].set_ylabel("Frecuencia")
    axes[1].set_title("Distribucion de Areas - Google Open Buildings")
    axes[1].axvline(
        x=areas.median(),
        color="blue",
        linestyle="--",
        label=f"Mediana: {areas.median():.1f} m2",
    )
    axes[1].legend()

plt.suptitle("Distribucion de Areas de Edificios en San Juan de Pasto", fontsize=14)
plt.tight_layout()
plt.savefig("./data/edificios_histograma_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 8. Deteccion de Edificios con Segment Anything Model (SAM)

SAM es un modelo de segmentacion de imagenes de proposito general desarrollado por Meta AI.

In [ ]:
# Nota: SAM requiere una imagen de entrada
# Primero necesitamos descargar una imagen satelital del area

# URL de imagen de ejemplo (Google Earth Engine Tile o similar)
# Para demostracion, usaremos la funcionalidad de samgeo

print("Configuracion de SAM para deteccion de edificios:")
print("1. Cargar imagen satelital del area")
print("2. Usar prompts de texto como 'building' o 'house'")
print("3. Generar mascaras de segmentacion")
print("4. Vectorizar los resultados")

In [ ]:
# Ejemplo de uso de SAM con GeoAI
# Este codigo requiere GPU para ejecutarse eficientemente

import torch

print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Configurar SAM (si hay GPU disponible)
# from samgeo import SamGeo

# sam = SamGeo(
#     model_type="vit_h",  # Modelo mas grande y preciso
#     checkpoint=None,  # Descargar automaticamente
#     device="cuda" if torch.cuda.is_available() else "cpu"
# )

# # Segmentar edificios usando prompts de texto
# sam.generate(
#     image_path,
#     output_path,
#     text_prompt="building"
# )

print("Nota: La deteccion con SAM requiere GPU para rendimiento optimo.")
print("En Colab, asegurate de seleccionar 'Runtime > Change runtime type > GPU'")

## 9. Densidad de Edificios

In [ ]:
# Calcular densidad de edificios por cuadricula
def crear_grilla(bbox, cell_size=0.002):
    """
    Crea una grilla sobre el bbox.
    cell_size en grados (aprox 200m a esta latitud)
    """
    minx, miny, maxx, maxy = bbox

    cells = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            cell = box(x, y, x + cell_size, y + cell_size)
            cells.append(cell)
            y += cell_size
        x += cell_size

    return gpd.GeoDataFrame(geometry=cells, crs="EPSG:4326")


# Crear grilla
grilla = crear_grilla(PASTO_BBOX, cell_size=0.003)
print(f"Celdas en la grilla: {len(grilla)}")

In [ ]:
# Calcular edificios por celda
if buildings_overture is not None and len(buildings_overture) > 0:
    # Contar edificios por celda
    edificios_por_celda = []
    for idx, celda in grilla.iterrows():
        count = buildings_overture[buildings_overture.intersects(celda.geometry)].shape[
            0
        ]
        edificios_por_celda.append(count)

    grilla["edificios"] = edificios_por_celda

    # Visualizar mapa de calor
    m_density = leafmap.Map(
        center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=15, height="600px"
    )

    m_density.add_basemap("Esri.WorldImagery")

    # Agregar grilla con colores segun densidad
    m_density.add_data(
        grilla, column="edificios", cmap="YlOrRd", layer_name="Densidad de Edificios"
    )

    m_density.add_layer_control()
    m_density

## 10. Guardar Resultados

In [ ]:
# Crear directorio de datos
os.makedirs("./data", exist_ok=True)

# Guardar edificios como GeoJSON
if buildings_overture is not None and len(buildings_overture) > 0:
    buildings_overture.to_file(
        "./data/edificios_overture_pasto.geojson", driver="GeoJSON"
    )
    print("Guardado: edificios_overture_pasto.geojson")

if buildings_google is not None and len(buildings_google) > 0:
    buildings_google.to_file("./data/edificios_google_pasto.geojson", driver="GeoJSON")
    print("Guardado: edificios_google_pasto.geojson")

# Guardar grilla de densidad
if "edificios" in grilla.columns:
    grilla.to_file("./data/densidad_edificios_pasto.geojson", driver="GeoJSON")
    print("Guardado: densidad_edificios_pasto.geojson")

## 11. Lugares Especificos de Pasto

In [ ]:
# Analizar edificios cerca de lugares especificos
lugares = {
    "Plaza de Narino": (-77.2811, 1.2136),
    "Universidad de Narino": (-77.2783, 1.2175),
    "Terminal de Transportes": (-77.2683, 1.2047),
    "Centro Comercial Unicentro": (-77.2731, 1.2003),
}

radio_metros = 200  # Radio de busqueda en metros

if buildings_overture is not None and len(buildings_overture) > 0:
    print("Edificios cerca de lugares importantes:")
    print("=" * 50)

    for nombre, (lon, lat) in lugares.items():
        # Crear buffer aproximado (en grados, ~200m)
        buffer_degrees = 0.002
        punto_buffer = box(
            lon - buffer_degrees,
            lat - buffer_degrees,
            lon + buffer_degrees,
            lat + buffer_degrees,
        )

        edificios_cercanos = buildings_overture[
            buildings_overture.intersects(punto_buffer)
        ]
        print(f"\n{nombre}:")
        print(f"  Edificios en radio ~200m: {len(edificios_cercanos)}")

## 12. Resumen

En este notebook hemos aprendido a:

1. **Descargar huellas de edificios** de Overture Maps y Google Open Buildings
2. **Visualizar edificios** en mapas interactivos
3. **Calcular estadisticas** de areas y cantidades
4. **Analizar la densidad** de construcciones en Pasto
5. **Preparar datos** para deteccion con SAM

### Conclusiones para Pasto:

- El centro historico tiene alta densidad de edificios pequeños
- Las zonas comerciales tienen edificios mas grandes
- Hay diferencias entre las fuentes de datos (Overture vs Google)

### Proximos pasos:

- Usar SAM para detectar edificios en imagenes de alta resolucion
- Entrenar modelos personalizados para la arquitectura de Pasto